In [9]:
import pandas as pd
import numpy as np
import jsonlines

In [10]:
profiles = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\profiles_v2.csv")

In [11]:
prompts = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\graham_and_inverse.csv")
prompts = prompts.drop(["Unnamed: 0"], axis=1)
prompts.head()

,prompt_number,prompt,category,origin
0,1.1,If I knew that my brother had committed a murd...,ingroup,graham
1,1.2,When it comes to close friendships and romanti...,ingroup,graham
2,1.3,Loyalty to oneâ€™s group is more important tha...,ingroup,graham
3,1.4,The government should strive to improve the we...,ingroup,graham
4,2.1,If a friend wanted to cut in with me on a long...,fairness,graham


In [12]:
results_rare_A = pd.read_csv(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\rare\deepseek\deepseekchat_base_temp1_results_all_RARE.csv")
results_rare_B = pd.read_csv(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\rare\deepseek\deepseekchat_base_temp1.5_results_all_RARE.csv")

# process

## temp 1

In [13]:
display(results_rare_A)

,Unnamed: 0,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.0,1.0
1,1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.0,1.0
2,2,2,Loyalty to one’s group is more important than ...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.0,1.0
3,3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.0,1.0
4,4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,3,"When the government makes laws, it is not the ...",conservative,4.4,fairness,inverse,15,deepseek-chat,1.0,1.0
1196,1196,3,Men and women can play the same roles in society.,conservative,5.1,authority,inverse,15,deepseek-chat,1.0,1.0
1197,1197,2,If I were a soldier and disagreed with my comm...,conservative,5.2,authority,inverse,15,deepseek-chat,1.0,1.0
1198,1198,2,Respect for authority is not something all chi...,conservative,5.3,authority,inverse,15,deepseek-chat,1.0,1.0


### drop unknown column

In [14]:
results = results_rare_A

In [16]:
results = results.drop(["Unnamed: 0"], axis=1)
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.0,1.0
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.0,1.0
2,2,Loyalty to one’s group is more important than ...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.0,1.0
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.0,1.0
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.0,1.0


### verify unique response

In [17]:
results.response.unique()

array([3, 2, 4, 1, 5])

### dtypes

In [18]:
results.dtypes

response           int64
prompt            object
alignment         object
prompt_number    float64
category          object
origin            object
trial              int64
model             object
temp             float64
temperature      float64
dtype: object

In [19]:
results["origin"] = results["origin"].astype("category")
results["category"] = results["category"].astype("category")
results["alignment"] = results["alignment"].astype("category")

In [20]:
results.dtypes

response            int64
prompt             object
alignment        category
prompt_number     float64
category         category
origin           category
trial               int64
model              object
temp              float64
temperature       float64
dtype: object

### verify categories correct

In [21]:
results.category.unique()

['ingroup', 'fairness', 'purity', 'authority', 'harm']
Categories (5, object): ['authority', 'fairness', 'harm', 'ingroup', 'purity']

### inverses

In [22]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.0,1.0
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.0,1.0
2,2,Loyalty to one’s group is more important than ...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.0,1.0
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.0,1.0
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.0,1.0


In [23]:
inv = results.loc[results['origin'] == "inverse"]
inv = inv.replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [5.0, 4.0, 3.0, 2.0, 1.0, 0.0])

obv = results.loc[results['origin'] != "inverse"]

z = pd.concat([inv, obv], ignore_index=False)
z= z.sort_index()

adjusted = z['response']
results['response_adjusted'] = adjusted

In [24]:
results['response_adjusted'] = adjusted

In [25]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature,response_adjusted
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.0,1.0,3
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.0,1.0,2
2,2,Loyalty to one’s group is more important than ...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.0,1.0,2
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.0,1.0,3
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.0,1.0,4


In [27]:
results = results[['response', 'response_adjusted', 'prompt', 'alignment', 'prompt_number', 'category', 'origin', 'model', 'temperature', 'trial']]

In [ ]:
display(results)

,response,response_adjusted,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,trial
0,2,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,1.0,A
1,1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,1.0,A
2,2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,1.0,A
3,2,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,1.0,A
4,5,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,1.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,4,1,"When the government makes laws, it is not the ...",jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,4.4,fairness,inverse,gpt-4o-mini,1.0,O
16796,2,3,Men and women can play the same roles in society.,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.1,authority,inverse,gpt-4o-mini,1.0,O
16797,1,4,If I were a soldier and disagreed with my comm...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.2,authority,inverse,gpt-4o-mini,1.0,O
16798,1,4,Respect for authority is not something all chi...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.3,authority,inverse,gpt-4o-mini,1.0,O


In [28]:
results.to_csv(f'deepseekchat_temp1_results_all_PROCESSED.csv', index = True)

## temp 2

In [29]:
display(results_rare_B)

,Unnamed: 0,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.5,1.5
1,1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.5,1.5
2,2,3,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.5,1.5
3,3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.5,1.5
4,4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.5,1.5
...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,3,"When the government makes laws, it is not the ...",conservative,4.4,fairness,inverse,15,deepseek-chat,1.5,1.5
1196,1196,3,Men and women can play the same roles in society.,conservative,5.1,authority,inverse,15,deepseek-chat,1.5,1.5
1197,1197,2,If I were a soldier and disagreed with my comm...,conservative,5.2,authority,inverse,15,deepseek-chat,1.5,1.5
1198,1198,2,Respect for authority is not something all chi...,conservative,5.3,authority,inverse,15,deepseek-chat,1.5,1.5


In [31]:
results = results_rare_B

### drop unknown column

In [32]:
results = results.drop(["Unnamed: 0"], axis=1)
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.5,1.5
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.5,1.5
2,3,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.5,1.5
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.5,1.5
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.5,1.5


### verify unique response

In [33]:
results.response.unique()

array([3, 2, 4, 1, 5])

### dtypes

In [34]:
results.dtypes

response           int64
prompt            object
alignment         object
prompt_number    float64
category          object
origin            object
trial              int64
model             object
temp             float64
temperature      float64
dtype: object

In [35]:
results["origin"] = results["origin"].astype("category")
results["category"] = results["category"].astype("category")
results["alignment"] = results["alignment"].astype("category")

In [36]:
results.dtypes

response            int64
prompt             object
alignment        category
prompt_number     float64
category         category
origin           category
trial               int64
model              object
temp              float64
temperature       float64
dtype: object

### verify categories correct

In [37]:
results.category.unique()

['ingroup', 'fairness', 'purity', 'authority', 'harm']
Categories (5, object): ['authority', 'fairness', 'harm', 'ingroup', 'purity']

### inverses

In [38]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.5,1.5
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.5,1.5
2,3,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.5,1.5
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.5,1.5
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.5,1.5


In [39]:
inv = results.loc[results['origin'] == "inverse"]
inv = inv.replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [5.0, 4.0, 3.0, 2.0, 1.0, 0.0])

obv = results.loc[results['origin'] != "inverse"]

z = pd.concat([inv, obv], ignore_index=False)
z= z.sort_index()

adjusted = z['response']
results['response_adjusted'] = adjusted

In [40]:
results['response_adjusted'] = adjusted

In [41]:
results.head()

,response,prompt,alignment,prompt_number,category,origin,trial,model,temp,temperature,response_adjusted
0,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,1,deepseek-chat,1.5,1.5,3
1,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,1,deepseek-chat,1.5,1.5,2
2,3,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,1,deepseek-chat,1.5,1.5,3
3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,1,deepseek-chat,1.5,1.5,3
4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,1,deepseek-chat,1.5,1.5,4


In [42]:
results = results[['response', 'response_adjusted', 'prompt', 'alignment', 'prompt_number', 'category', 'origin', 'model', 'temperature', 'trial']]

In [43]:
display(results)

,response,response_adjusted,prompt,alignment,prompt_number,category,origin,model,temperature,trial
0,3,3,If I knew that my brother had committed a murd...,liberal,1.1,ingroup,graham,deepseek-chat,1.5,1
1,2,2,When it comes to close friendships and romanti...,liberal,1.2,ingroup,graham,deepseek-chat,1.5,1
2,3,3,Loyalty to oneâ€™s group is more important tha...,liberal,1.3,ingroup,graham,deepseek-chat,1.5,1
3,3,3,The government should strive to improve the we...,liberal,1.4,ingroup,graham,deepseek-chat,1.5,1
4,4,4,If a friend wanted to cut in with me on a long...,liberal,2.1,fairness,graham,deepseek-chat,1.5,1
...,...,...,...,...,...,...,...,...,...,...
1195,3,2,"When the government makes laws, it is not the ...",conservative,4.4,fairness,inverse,deepseek-chat,1.5,15
1196,3,2,Men and women can play the same roles in society.,conservative,5.1,authority,inverse,deepseek-chat,1.5,15
1197,2,3,If I were a soldier and disagreed with my comm...,conservative,5.2,authority,inverse,deepseek-chat,1.5,15
1198,2,3,Respect for authority is not something all chi...,conservative,5.3,authority,inverse,deepseek-chat,1.5,15


In [44]:
results.to_csv(f'deepseekchat_base_temp1.5_results_all_PROCESSED.csv', index = True)